In [1]:
import pymysql
print("pymysql is installed and working!")



pymysql is installed and working!


In [2]:
import pymysql

# MySQL connection details
DATABASE_USER = "root"
DATABASE_PASSWORD = "bokang13"  
DATABASE_NAME = "bigdata_project"

# Connect to MySQL
connection = pymysql.connect(
    host="localhost",
    user=DATABASE_USER,
    password=DATABASE_PASSWORD
)

# Create the database
query = "CREATE DATABASE IF NOT EXISTS bigdata_project;"
cursor = connection.cursor()
cursor.execute(query)
connection.commit()
cursor.close()
connection.close()

print("database Created Successfully!")

database Created Successfully!


In [3]:
# Reconnect to MySQL
connection = pymysql.connect(
    host="localhost",
    user=DATABASE_USER,
    password=DATABASE_PASSWORD,
    database=DATABASE_NAME
)

print(" Switched to bigdata_project!")


 Switched to bigdata_project!


In [ ]:
import pandas as pd

books_reviews = pd.read_csv("Books_rating.csv", usecols=["Id", "Title", "User_id", "review/score", "review/time"])
books_details = pd.read_csv("books_data.csv", usecols=["Title", "categories"])
books_reviews["review/time"] = pd.to_datetime(books_reviews["review/time"], unit='s')

books_merged = books_reviews.merge(books_details, on="Title", how="left")

books_merged.rename(columns={
    "Id": "book_id",
    "review/score": "review_score",
    "review/time": "review_date",
    "categories": "genre"
}, inplace=True)

books_merged = books_merged.assign(genre=books_merged["genre"].fillna("Unknown"))
print("Data cleaned successfully!")

In [4]:
import pandas as pd

books_merged = pd.read_csv("books_cleaned.csv")
print("Merged file loaded successfully!")
display(books_merged.head())


Merged file loaded successfully!


,book_id,Title,User_id,review_score,review_date,genre
0,1882931173,Its Only Art If Its Well Hung!,AVCGYZL8FQQTD,4.0,1999-10-23,['Comics & Graphic Novels']
1,0826414346,Dr. Seuss: American Icon,A30TK6U7DNS82R,5.0,2004-09-21,['Biography & Autobiography']
2,0826414346,Dr. Seuss: American Icon,A3UH4UZ4RSVO82,5.0,2004-03-09,['Biography & Autobiography']
3,0826414346,Dr. Seuss: American Icon,A2MVUWT453QH61,4.0,2004-07-25,['Biography & Autobiography']
4,0826414346,Dr. Seuss: American Icon,A22X4XUPKF66MR,4.0,2005-02-10,['Biography & Autobiography']


In [6]:
books_merged = books_merged.fillna({
    "book_id": "Unknown",
    "Title": "Unknown",
    "User_id": "Unknown",
    "review_score": 0.0,
    "review_date": "0000-00-00",
    "genre": "Unknown"
})
print("Missing values filled!")


Missing values filled!


In [7]:
import pymysql

connection = pymysql.connect(
    host="localhost",
    user="root",
    password="bokang13",
    database="bigdata_project"
)
cursor = connection.cursor()

query = """
CREATE TABLE IF NOT EXISTS book_reviews (
    book_id VARCHAR(20) PRIMARY KEY,
    title VARCHAR(500),
    user_id VARCHAR(50),
    review_score FLOAT,
    review_date DATE,
    genre VARCHAR(500)
);
"""
cursor.execute(query)
connection.commit()
print(" Table created or already exists!")


 Table created or already exists!


In [8]:
for _, row in books_merged.iterrows():
    query = """
    INSERT IGNORE INTO book_reviews (book_id, title, user_id, review_score, review_date, genre)
    VALUES (%s, %s, %s, %s, %s, %s)
    """
    values = (
        row["book_id"],
        row["Title"],
        row["User_id"],
        float(row["review_score"]),
        row["review_date"],
        row["genre"][:500]
    )
    cursor.execute(query, values)

connection.commit()
cursor.close()
connection.close()
print(" Data inserted successfully into MySQL!")


 Data inserted successfully into MySQL!


In [11]:

import pandas as pd
import time
import pymysql


connection = pymysql.connect(
    host="localhost",
    user="root",
    password="bokang13",  
    database="bigdata_project"
)
cursor = connection.cursor()


start_time = time.time()

query = """
SELECT *
FROM (
    SELECT 
        genre,
        ROUND(AVG(review_score), 2) AS avg_rating,
        COUNT(*) AS review_count
    FROM 
        book_reviews
    GROUP BY 
        genre
    ORDER BY 
        review_count DESC
    LIMIT 50
) AS top_50_genres
ORDER BY avg_rating DESC
LIMIT 10;

"""

df = pd.read_sql(query, con=connection)
display(df)

end_time = time.time()
print(f"Execution Time: {end_time - start_time:.4f} seconds")



/var/folders/86/h4mb42gn6j12gvzvy1rntg2h0000gn/T/ipykernel_34557/1860114749.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=connection)


,genre,avg_rating,review_count
0,['Adventure stories'],4.55,14758
1,['Juvenile Fiction'],4.45,415924
2,"[""Children's stories""]",4.43,10808
3,['Poetry'],4.42,20566
4,['Juvenile Nonfiction'],4.42,58704
5,['Bibles'],4.40,11626
6,['Cooking'],4.40,59836
7,['Literary Criticism'],4.39,34580
8,['Self-Help'],4.39,58166
9,['Drama'],4.39,28788


Execution Time: 5.0043 seconds


In [15]:
import pandas as pd

movies_merged = pd.read_csv("movies_cleaned.csv")
print("Merged file loaded successfully!")
display(movies_merged.head())

Merged file loaded successfully!


,movie_id,reviewId,review_date,review_score,movie_title,genre
0,beavers,1145982,2003-05-23,3.5,Beavers,Documentary
1,blood_mask,1636744,2007-06-02,1.0,Blood Mask,Unknown
2,city_hunter_shinjuku_private_eyes,2590987,2019-05-28,NaN,City Hunter: Shinjuku Private Eyes,Unknown
3,city_hunter_shinjuku_private_eyes,2558908,2019-02-14,2.5,City Hunter: Shinjuku Private Eyes,Unknown
4,dangerous_men_2015,2504681,2018-08-29,NaN,Dangerous Men,Unknown


In [16]:
import pymysql

# Connect to MySQL
connection = pymysql.connect(
    host="localhost",
    user="root",
    password="bokang13",
    database="bigdata_project"
)

cursor = connection.cursor()


cursor.execute("DROP TABLE IF EXISTS movie_reviews;")

# Create new table
query = """
CREATE TABLE movie_reviews (
    review_id INT PRIMARY KEY,
    movie_id VARCHAR(255),
    movie_title VARCHAR(255),
    review_date DATE,
    review_score FLOAT,
    genre VARCHAR(500)
);
"""
cursor.execute(query)
connection.commit()

print("Movies table created successfully")



Movies table created successfully


In [25]:
import pymysql
import numpy as np

# Connect to MySQL
connection = pymysql.connect(
    host="localhost",
    user="root",
    password="bokang13",
    database="bigdata_project"
)

cursor = connection.cursor()

# Final up unknowns
movies_merged = movies_merged.fillna({
    "reviewId": 0,
    "movie_id": "Unknown",
    "movie_title": "Unknown",
    "review_date": "0000-00-00",
    "review_score": 0.0,
    "genre": "Unknown"
})


# Insert into MySQL
for _, row in movies_merged.iterrows():
    query = """
    INSERT IGNORE INTO movie_reviews (review_id, movie_id, movie_title, review_date, review_score, genre)
    VALUES (%s, %s, %s, %s, %s, %s)
    """
    values = (
        row["reviewId"],
        row["movie_id"],
        row["movie_title"],
        row["review_date"],
        row["review_score"],
        row["genre"][:500]
    )
    cursor.execute(query, values)

# Commit & close
connection.commit()
cursor.close()
connection.close()

print("Data inserted successfully!")


Data inserted successfully!


In [44]:
import pymysql
import time
import pandas as pd

# Reconnect to MySQLletlet
connection = pymysql.connect(
    host="localhost",
    user="root",
    password="bokang13",  
    database="bigdata_project"
)

start_time = time.time()

query = """
SELECT 
    genre,
    ROUND(AVG(review_score), 2) AS avg_rating,
    COUNT(*) AS review_count
FROM 
    movie_reviews
GROUP BY 
    genre
ORDER BY 
    review_count DESC,
    avg_rating DESC
LIMIT 10;
"""

df = pd.read_sql(query, con=connection)
display(df)

end_time = time.time()
print(f"Execution Time: { end_time:.4f} seconds")


connection.close()



/var/folders/86/h4mb42gn6j12gvzvy1rntg2h0000gn/T/ipykernel_34557/2435389803.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=connection)


,genre,avg_rating,review_count
0,Drama,2.46,206844
1,Comedy,2.24,101884
2,Documentary,2.28,96191
3,"Comedy, Drama",2.45,82588
4,"Horror, Mystery & thriller",2.19,42474
5,"Romance, Comedy",2.18,42416
6,Mystery & thriller,2.44,39735
7,Horror,2.39,26384
8,Unknown,2.34,23152
9,"Crime, Drama",2.74,22394


Execution Time: 60.5030 seconds
